In [1]:
import os
import glob
import numpy as np
import SimpleITK as sitk
import seaborn as sns
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
reg_path = '/Users/mona/GitClone/voxelmorph-test/data/data_moved'
orig_path = '/Users/mona/GitClone/voxelmorph-test/data/data_test'
output_img = '/Users/mona/GitClone/voxelmorph-test/data/output_img'

orig = sorted(glob.glob(f"{orig_path}/*.npy"))
registed = sorted(glob.glob(f"{reg_path}/*.nii"))

In [ ]:
def pca(data):
    # calculate the eigenvalues of data(covariance matrix)
    array = np.transpose(np.load(orig[0]), (2,1,0))
    x, y, z = array.shape
    M = array.reshape(x*y, z)
    Sigma = np.diag(np.std(M, axis=0))
    M_avg = np.tile(np.average(M, axis=0), (x*y, 1))
    K = np.dot(np.dot(np.dot(Sigma.T, (M - M_avg).T), (M - M_avg)), Sigma.T) / (x*y - 1)
    eigenvalues = np.linalg.eigvals(K)
    return eigenvalues

In [3]:
tests = []
org_mse = []
rig_mse = []
for i in range(len(registed)):
    rig = sitk.GetArrayFromImage(sitk.ReadImage(registed[i]))
    org = np.transpose(np.load(orig[i]), (2,1,0))
    # print(f"original shape {org.shape} and rigestered shape {rig.shape}")
    loss_rig, loss_org = 0, 0
    name = (registed[i]).split("/")[-1]
    for j in range(1, rig.shape[-1]):
        loss_rig += mse(rig[:,:,j-1], rig[:,:,j])
        loss_org += mse(org[:,:,j-1], org[:,:,j])
    print(f"File {name}, mse of original image is {loss_org}, mse of registered image is {loss_rig}")
    org_mse.append(loss_org)
    rig_mse.append(loss_rig)
    tests.append(name)

    eig_org = pca(org)
    eig_rig = pca(rig)

    f, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
    sns.barplot(x=np.arange(len(eig_org)), y=eig_org, palette="rocket", ax=ax1)
    sns.barplot(x=np.arange(len(eig_rig)), y=eig_rig, palette="rocket", ax=ax2)
    ax1.set_title(f"Eigenvalues of original image {name}")
    plt.savefig(os.path.join(output_img, f"{name[:-4]}_pca_barplot.png"))


File 0003_rawimages_slice_0.nii, mse of original image is 0.006164561102406497, mse of registered image is 0.002939408198205723
File 0004_rawimages_slice_2.nii, mse of original image is 0.008026560705277264, mse of registered image is 0.0034939513811973774
File 0009_rawimages_slice_1.nii, mse of original image is 0.017407262249532984, mse of registered image is 0.006282985956542479
File 0010_rawimages_slice_2.nii, mse of original image is 0.010713342084593873, mse of registered image is 0.004520152397549455
File 0015_rawimages_slice_2.nii, mse of original image is 0.00483511294099898, mse of registered image is 0.0020940793671900926
File 0029_rawimages_slice_0.nii, mse of original image is 0.03256052963813195, mse of registered image is 0.009602719299245076
File 0030_rawimages_slice_2.nii, mse of original image is 0.03381547427112286, mse of registered image is 0.01142995365420301
File 0031_rawimages_slice_1.nii, mse of original image is 0.03037667223357461, mse of registered image is 

In [11]:
import pandas as pd
df = pd.DataFrame({'Cases': tests, 'raw MSE': org_mse, 'registered MSE': rig_mse})
# df = pd.DataFrame()

In [12]:
def percentage_change(col1,col2):
    return ((col2 - col1) / col1) * 100

df['MSE changes percentage'] = percentage_change(df['raw MSE'], df['registered MSE'])
df

,Cases,raw MSE,registered MSE,MSE changes percentage
0,0003_rawimages_slice_0.nii,0.006165,0.002939,-52.317640
1,0004_rawimages_slice_2.nii,0.008027,0.003494,-56.470131
2,0009_rawimages_slice_1.nii,0.017407,0.006283,-63.905950
3,0010_rawimages_slice_2.nii,0.010713,0.004520,-57.808195
4,0015_rawimages_slice_2.nii,0.004835,0.002094,-56.690166
5,0029_rawimages_slice_0.nii,0.032561,0.009603,-70.508099
6,0030_rawimages_slice_2.nii,0.033815,0.011430,-66.199044
7,0031_rawimages_slice_1.nii,0.030377,0.014845,-51.130992
8,0032_rawimages_slice_2.nii,0.038394,0.015576,-59.430314
9,0033_rawimages_slice_2.nii,0.040512,0.013882,-65.732768
